In [37]:
file_path = r"C:\Users\Shirley\Downloads\BGL\BGL.log"

with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
    for i in range(100):
        print(f.readline().strip())


- 1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.363779 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected
- 1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.527847 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected
- 1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.675872 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected
- 1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.823719 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected
- 1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.982731 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected
- 1117838571 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.51.131467 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected
- 1117838571 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.51.293532 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instr

In [3]:
pip uninstall logparser -y


Found existing installation: logparser 0.8.4
Uninstalling logparser-0.8.4:
  Successfully uninstalled logparser-0.8.4
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install git+https://github.com/logpai/logparser.git


In [2]:
from logparser import Drain


In [3]:
from logparser.Drain import LogParser
print("Drain parser loaded successfully!")


Drain parser loaded successfully!


In [4]:
import os

In [5]:
import re

In [14]:
##preprocessing before parsing##
input_path = r"C:\Users\Shirley\Downloads\BGL\BGL.log"
output_path = r"C:\Users\Shirley\Downloads\BGL\BGL_clean.log"
with open(input_path, "r", encoding="utf-8", errors="ignore") as infile, \
     open(output_path, "w", encoding="utf-8") as outfile:
    
    for line in infile:
        line = line.strip()
        if not line:
            continue
        
        # remove"-" and extra space
        line = re.sub(r"^-+\s*", "", line)
        
        #  extract "RAS" or "INFO" important contents
        match = re.search(r"(RAS.*)", line)
        if match:
            outfile.write(match.group(1) + "\n")

In [15]:
import os
output_path = r"C:\Users\Shirley\Downloads\BGL\BGL_clean.log"
print("file exist：", os.path.exists(output_path))


file exist： True


In [16]:
with open(output_path, "r", encoding="utf-8") as f:
    for i in range(10):
        print(f.readline().strip())


RAS KERNEL INFO instruction cache parity error corrected
RAS KERNEL INFO instruction cache parity error corrected
RAS KERNEL INFO instruction cache parity error corrected
RAS KERNEL INFO instruction cache parity error corrected
RAS KERNEL INFO instruction cache parity error corrected
RAS KERNEL INFO instruction cache parity error corrected
RAS KERNEL INFO instruction cache parity error corrected
RAS KERNEL INFO instruction cache parity error corrected
RAS KERNEL INFO instruction cache parity error corrected
RAS KERNEL INFO instruction cache parity error corrected


In [17]:
##simplyly check the preprocessed data##
with open(output_path, "r", encoding="utf-8") as f:
    lines = sum(1 for _ in f)
print(f"there are {lines:,} after cleaning.")


there are 4,643,748 after cleaning.


In [8]:
from logparser.Drain import LogParser
import os

In [30]:
with open(r"C:\Users\Shirley\Downloads\BGL\BGL.log", 'r', encoding='utf-8', errors='ignore') as f:
    for i in range(5):
        print(f.readline())


- 1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.363779 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected

- 1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.527847 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected

- 1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.675872 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected

- 1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.823719 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected

- 1117838570 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.42.50.982731 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected



In [20]:
# SimpleDrain: 一个从头实现的轻量级 Drain-style parser
# 直接适配你已经生成的 BGL_clean.log
import os
import re
import pandas as pd
from collections import defaultdict
from hashlib import md5
from tqdm import tqdm

class SimpleDrain:
    def __init__(self, st=0.4, max_templates=None, verbose=True):
        """
        st: 相似度阈值 (0..1)，越大越严格（更难合并成同一模板）
        max_templates: 可选，限制模板数量（防止爆炸），None 表示无限制
        """
        self.st = st
        self.max_templates = max_templates
        self.templates = []        # list of token lists (模板)
        self.template_counts = []  # 对应出现次数
        self.log_to_template = []  # 每条日志映射到模板索引
        self.verbose = verbose

    def tokenize(self, line):
        # 简单按空格分词，同时把多空格压缩
        tokens = re.split(r"\s+", line.strip())
        return tokens

    def similarity(self, seq1, seq2):
        """
        计算两个 token 序列的相似度。
        采用匹配位置相同 token 的比例，归一化除以两序列的最大长度，
        这样能兼容不同长度的日志行。
        """
        if not seq1 or not seq2:
            return 0.0
        L = max(len(seq1), len(seq2))
        same = 0
        # 对齐前 min_len 部分进行比较（按索引）
        min_len = min(len(seq1), len(seq2))
        for i in range(min_len):
            if seq1[i] == seq2[i] or seq1[i] == "<*>" or seq2[i] == "<*>":
                same += 1
        # 长度差异视为不同位
        return same / L

    def merge_template(self, temp, tokens):
        """
        将已有模板和新tokens合并成更泛化的模板（不同位置替换为 <*>）。
        输出长度为 max(len(temp), len(tokens))。
        """
        L = max(len(temp), len(tokens))
        merged = []
        for i in range(L):
            t = temp[i] if i < len(temp) else "<*>"
            s = tokens[i] if i < len(tokens) else "<*>"
            merged.append(t if t == s else "<*>")
        return merged

    def add_log(self, tokens):
        """
        将一个 token 列表加入到已有模板中或创建新模板。
        返回模板索引。
        """
        best_sim = -1.0
        best_idx = None
        # 遍历模板找最相似的
        for idx, temp in enumerate(self.templates):
            sim = self.similarity(temp, tokens)
            if sim > best_sim:
                best_sim = sim
                best_idx = idx

        if best_sim >= self.st and best_idx is not None:
            # 合并模板并更新计数
            new_temp = self.merge_template(self.templates[best_idx], tokens)
            self.templates[best_idx] = new_temp
            self.template_counts[best_idx] += 1
            return best_idx
        else:
            # 创建新模板
            if self.max_templates is not None and len(self.templates) >= self.max_templates:
                # 如果达到上限，放入最相似的模板（即使不达阈值）
                self.template_counts[best_idx] += 1
                new_index = best_idx
                # 仍然尝试合并以避免太多模板
                self.templates[new_index] = self.merge_template(self.templates[new_index], tokens)
                return new_index
            else:
                self.templates.append(tokens)
                self.template_counts.append(1)
                return len(self.templates) - 1

    def parse_lines(self, lines):
        """
        主入口：解析多行日志（list of strings）。
        会填充 self.log_to_template。
        """
        self.log_to_template = []
        iterator = tqdm(lines, desc="Parsing logs") if self.verbose else lines
        for line in iterator:
            tokens = self.tokenize(line)
            idx = self.add_log(tokens)
            self.log_to_template.append(idx)

    def save_results(self, lines, output_dir, base_name="BGL"):
        """
        保存结果到 output_dir：
          - {base_name}_structured.csv  (LineId, Content, EventTemplate, EventId)
          - {base_name}_templates.csv   (EventId, EventTemplate, Occurrences)
        """
        os.makedirs(output_dir, exist_ok=True)
        records = []
        for i, line in enumerate(lines):
            tidx = self.log_to_template[i]
            template_tokens = self.templates[tidx]
            template_str = " ".join(template_tokens)
            event_id = md5(template_str.encode("utf-8")).hexdigest()[:8]
            records.append({
                "LineId": i + 1,
                "Content": line,
                "EventTemplate": template_str,
                "EventId": event_id
            })
        df_struct = pd.DataFrame(records)
        out_struct = os.path.join(output_dir, f"{base_name}_structured.csv")
        df_struct.to_csv(out_struct, index=False, encoding="utf-8")
        if self.verbose:
            print("Saved structured logs:", out_struct)

        # templates summary
        temps = []
        for i, temp in enumerate(self.templates):
            temp_str = " ".join(temp)
            event_id = md5(temp_str.encode("utf-8")).hexdigest()[:8]
            occ = self.template_counts[i]
            temps.append({"EventId": event_id, "EventTemplate": temp_str, "Occurrences": occ})
        df_temp = pd.DataFrame(temps).sort_values(by="Occurrences", ascending=False)
        out_temp = os.path.join(output_dir, f"{base_name}_templates.csv")
        df_temp.to_csv(out_temp, index=False, encoding="utf-8")
        if self.verbose:
            print("Saved templates summary:", out_temp)

# ---------------------------
# ---------- 使用示例 ----------
# ---------------------------

input_file = r"C:\Users\Shirley\Downloads\BGL\BGL_clean.log"
output_dir = r"C:\Users\Shirley\Downloads\BGL\result_custom"
base_name = "BGL_clean"

# 读取日志（可以考虑抽样以加速调试）
with open(input_file, "r", encoding="utf-8", errors="ignore") as f:
    lines = [ln.strip() for ln in f if ln.strip()]

print("Total cleaned lines:", len(lines))

# 如果日志太大，先只处理前 20000 行来调试
sample_n = 50000  # 你可以改成 None 表示全部，或改小如 20000
if sample_n is not None and sample_n < len(lines):
    print(f"Using first {sample_n} lines for parsing (adjust sample_n to use more).")
    lines_proc = lines[:sample_n]
else:
    lines_proc = lines

# 初始化 parser（st 可以调整，例如 0.4/0.5/0.6）
parser = SimpleDrain(st=0.5, max_templates=None, verbose=True)

# 解析
parser.parse_lines(lines_proc)

# 保存结果
parser.save_results(lines_proc, output_dir, base_name=base_name)

# 简单检查：打印前 10 个模板
import pandas as pd
df_temp = pd.read_csv(os.path.join(output_dir, f"{base_name}_templates.csv"))
print("\nTop 10 templates:")
print(df_temp.head(10).to_string(index=False))


Total cleaned lines: 4643748
Using first 50000 lines for parsing (adjust sample_n to use more).


Parsing logs: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [00:04<00:00, 11657.46it/s]


Saved structured logs: C:\Users\Shirley\Downloads\BGL\result_custom\BGL_clean_structured.csv
Saved templates summary: C:\Users\Shirley\Downloads\BGL\result_custom\BGL_clean_templates.csv

Top 10 templates:
 EventId                                                                                     EventTemplate  Occurrences
ffa11f25                                                            RAS KERNEL <*> <*> <*> <*> <*> <*> <*>        31859
0a9d14eb                                   RAS <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*> <*>        11377
b80f2820                                          RAS KERNEL INFO instruction cache parity error corrected         4867
62462bfa                                                        RAS KERNEL INFO CE sym <*> at <*> mask <*>         1175
27f8f364                                                         RAS KERNEL FATAL data TLB error interrupt          267
284519f8 RAS APP FATAL ciod: Error loading <*> invalid or missing program 

In [21]:
import re
import os
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
import hashlib
from datetime import datetime

# === Simple implementation of Drain-like log parser ===
class SimpleDrain:
    def __init__(self, st=0.4, depth=4):
        """
        st: similarity threshold between templates
        depth: maximum token depth for template tree
        """
        self.st = st
        self.depth = depth
        self.templates = []
        self.event_ids = []

    def _similarity(self, seq1, seq2):
        """Compute similarity between two token sequences"""
        same = sum(t1 == t2 for t1, t2 in zip(seq1, seq2))
        return same / max(len(seq1), len(seq2))

    def fit_transform(self, logs):
        """Cluster logs into templates"""
        structured = []
        for log in tqdm(logs, desc="Parsing logs"):
            tokens = log.split()
            matched = False
            for tid, template in enumerate(self.templates):
                if len(tokens) != len(template):
                    continue
                sim = self._similarity(tokens, template)
                if sim >= self.st:
                    # merge templates
                    new_template = [
                        t1 if t1 == t2 else "<*>" for t1, t2 in zip(template, tokens)
                    ]
                    self.templates[tid] = new_template
                    structured.append((tid, " ".join(new_template)))
                    matched = True
                    break
            if not matched:
                # create new template
                self.templates.append(tokens)
                structured.append((len(self.templates) - 1, " ".join(tokens)))
        return structured

    def save_results(self, structured, output_dir):
        """Save structured logs and template summary"""
        os.makedirs(output_dir, exist_ok=True)
        log_df = pd.DataFrame(structured, columns=["TemplateID", "EventTemplate"])
        log_df["EventId"] = log_df["EventTemplate"].map(
            lambda x: hashlib.md5(x.encode("utf-8")).hexdigest()[0:8]
        )
        log_df["Occurrences"] = log_df.groupby("EventId")["EventId"].transform("count")

        log_df.to_csv(os.path.join(output_dir, "BGL_structured.csv"), index=False)

        # Generate summary of unique templates
        template_summary = (
            log_df[["EventId", "EventTemplate", "Occurrences"]]
            .drop_duplicates()
            .sort_values(by="Occurrences", ascending=False)
        )
        template_summary.to_csv(
            os.path.join(output_dir, "BGL_templates.csv"), index=False
        )

        print(f"✅ Saved structured logs: {os.path.join(output_dir, 'BGL_structured.csv')}")
        print(f"✅ Saved template summary: {os.path.join(output_dir, 'BGL_templates.csv')}")

# === Step 1: Load cleaned logs ===
cleaned_log_path = r"C:\Users\Shirley\Downloads\BGL\BGL_clean.log"
output_dir = r"C:\Users\Shirley\Downloads\BGL\result_full"

with open(cleaned_log_path, "r", encoding="utf-8") as f:
    logs = [line.strip() for line in f if line.strip()]
print(f"Total cleaned log lines: {len(logs):,}")

# === Step 2: Run parsing on the full dataset ===
parser = SimpleDrain(st=0.4, depth=4)
start_time = datetime.now()
structured = parser.fit_transform(logs)
parser.save_results(structured, output_dir)
print(f"🕒 Total parsing time: {datetime.now() - start_time}")


Total cleaned log lines: 4,643,748


Parsing logs: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4643748/4643748 [02:45<00:00, 28103.17it/s]


✅ Saved structured logs: C:\Users\Shirley\Downloads\BGL\result_full\BGL_structured.csv
✅ Saved template summary: C:\Users\Shirley\Downloads\BGL\result_full\BGL_templates.csv
🕒 Total parsing time: 0:03:34.208087


In [41]:
import pandas as pd

df = pd.read_csv(r"C:\Users\Shirley\Downloads\BGL\result_full\BGL_structured.csv")
templates = pd.read_csv(r"C:\Users\Shirley\Downloads\BGL\result_full\BGL_templates.csv")

print(df.head())
print(df.columns)
print("Unique EventId:", df["EventId"].nunique())
print("Top 10 EventTemplate:")
print(templates.head(10))


   TemplateID                                      EventTemplate   EventId  \
0           0  RAS KERNEL INFO instruction cache parity error...  b80f2820   
1           0  RAS KERNEL INFO instruction cache parity error...  b80f2820   
2           0  RAS KERNEL INFO instruction cache parity error...  b80f2820   
3           0  RAS KERNEL INFO instruction cache parity error...  b80f2820   
4           0  RAS KERNEL INFO instruction cache parity error...  b80f2820   

   Occurrences  
0       105924  
1       105924  
2       105924  
3       105924  
4       105924  
Index(['TemplateID', 'EventTemplate', 'EventId', 'Occurrences'], dtype='object')
Unique EventId: 353
Top 10 EventTemplate:
    EventId                                      EventTemplate  Occurrences
0  45dd5525                             RAS KERNEL <*> <*> <*>      1712282
1  a985b82f                    RAS KERNEL INFO <*> <*> <*> <*>       756449
2  ffd19bff                   RAS KERNEL FATAL <*> <*> <*> <*>       268518
3 

In [27]:
import os
bgl_dir = r"C:\Users\Shirley\Downloads\BGL"

print("Files in BGL folder:")
for f in os.listdir(bgl_dir):
    print("-", f)


Files in BGL folder:
- anaconda_projects
- BGL.log
- BGL_clean.log
- result_custom
- result_full


In [38]:
import pandas as pd
df = pd.read_csv(r"C:\Users\Shirley\Downloads\BGL\result_full\BGL_structured.csv")
print(df.columns)


Index(['TemplateID', 'EventTemplate', 'EventId', 'Occurrences'], dtype='object')


In [39]:
import os
notebook_path = os.path.abspath("bgl_analysis.ipynb")
print(notebook_path)


C:\Windows\System32\bgl_analysis.ipynb


In [35]:
import pandas as pd

# 读取 BGL 日志文件的前几列（只要第一列即可）
file_path = r"C:\Users\Shirley\Downloads\BGL\BGL.log"

first_col_values = []

with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
    for line in f:
        parts = line.strip().split()
        if parts:
            first_col_values.append(parts[0])

# 转为 DataFrame 方便统计
df = pd.DataFrame(first_col_values, columns=['FirstColumn'])

# 统计不同值的出现次数
value_counts = df['FirstColumn'].value_counts()

print(value_counts.head(20))


FirstColumn
-            4399503
KERNDTLB      152734
KERNSTOR       63491
APPSEV         49651
KERNMNTF       31531
KERNTERM       23338
KERNREC         6145
APPREAD         5983
KERNRTSP        3983
APPRES          2370
APPUNAV         2048
APPTO           1991
KERNMICRO       1503
APPOUT           816
KERNMNT          720
APPBUSY          512
KERNMC           342
APPCHILD         320
KERNSOCK         209
KERNPOW          192
Name: count, dtype: int64


In [46]:
import pandas as pd

structured_path = r"C:\Users\Shirley\Downloads\BGL\result_full\BGL_structured.csv"
raw_path = r"C:\Users\Shirley\Downloads\BGL\BGL.log"

structured = pd.read_csv(structured_path)

# read the first column
raw = []
with open(raw_path, 'r', encoding='utf-8', errors='ignore') as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) > 0:
            raw.append(parts[0])

print(f"✅ Raw lines: {len(raw)}, Structured lines: {len(structured)}")

# Alignment Length
min_len = min(len(raw), len(structured))
if len(raw) != len(structured):
    print(f"⚠️ Length mismatch detected — truncating to {min_len} rows")

raw = raw[:min_len]
structured = structured.iloc[:min_len]

# add binary label
structured['FirstColumn'] = raw
structured['Label'] = structured['FirstColumn'].apply(lambda x: 0 if x == '-' else 1)

# save it
output_path = r"C:\Users\Shirley\Downloads\BGL\result_full\BGL_structured_labeled.csv"
structured.to_csv(output_path, index=False)

print(f"✅ Saved labeled data to: {output_path}")
print(structured['Label'].value_counts())


✅ Raw lines: 4747963, Structured lines: 4643748
⚠️ Length mismatch detected — truncating to 4643748 rows
✅ Saved labeled data to: C:\Users\Shirley\Downloads\BGL\result_full\BGL_structured_labeled.csv
Label
0    4308712
1     335036
Name: count, dtype: int64


In [47]:
file_path = r"C:\Users\Shirley\Downloads\BGL\result_full\BGL_structured_labeled.csv"

with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
    for i in range(100):
        print(f.readline().strip())

TemplateID,EventTemplate,EventId,Occurrences,FirstColumn,Label
0,RAS KERNEL INFO instruction cache parity error corrected,b80f2820,105924,-,0
0,RAS KERNEL INFO instruction cache parity error corrected,b80f2820,105924,-,0
0,RAS KERNEL INFO instruction cache parity error corrected,b80f2820,105924,-,0
0,RAS KERNEL INFO instruction cache parity error corrected,b80f2820,105924,-,0
0,RAS KERNEL INFO instruction cache parity error corrected,b80f2820,105924,-,0
0,RAS KERNEL INFO instruction cache parity error corrected,b80f2820,105924,-,0
0,RAS KERNEL INFO instruction cache parity error corrected,b80f2820,105924,-,0
0,RAS KERNEL INFO instruction cache parity error corrected,b80f2820,105924,-,0
0,RAS KERNEL INFO instruction cache parity error corrected,b80f2820,105924,-,0
0,RAS KERNEL INFO instruction cache parity error corrected,b80f2820,105924,-,0
0,RAS KERNEL INFO instruction cache parity error corrected,b80f2820,105924,-,0
0,RAS KERNEL INFO instruction cache parity error corrected,b80f2820,

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse as sp
import numpy as np

# ================================
# 1. Load structured + labeled log
# ================================
input_path = r"C:\Users\Shirley\Downloads\BGL\result_full\BGL_structured_labeled.csv"
df = pd.read_csv(input_path)

print("✅ Loaded data:", df.shape)
print(df.head(3))

# ======================================
# 2. Prepare text and labels for TF-IDF
# ======================================
# Use the 'EventTemplate' column as the textual content
# Labels (0=normal, 1=anomaly) will be kept for later training
texts = df["EventTemplate"].astype(str)
labels = df["Label"].astype(int)

# =====================================
# 3. TF-IDF feature extraction
# =====================================
# You can adjust max_features depending on memory (e.g. 1000–10000)
vectorizer = TfidfVectorizer(max_features=5000)

# Fit and transform the EventTemplate text
X_tfidf = vectorizer.fit_transform(texts)

print("✅ TF-IDF matrix shape:", X_tfidf.shape)

# =====================================
# 4. Save the TF-IDF features and labels
# =====================================
# Option 1: Save as compressed sparse matrix (.npz)
output_npz = r"C:\Users\Shirley\Downloads\BGL\result_full\BGL_TFIDF_features.npz"
sp.save_npz(output_npz, X_tfidf)

# Also save the labels as a separate file
np.save(r"C:\Users\Shirley\Downloads\BGL\result_full\BGL_labels.npy", labels)

# Option 2 (optional): Save a small preview CSV (for inspection only)
# ⚠️ Only recommended for a small subset, as full TF-IDF can be huge
preview_csv = r"C:\Users\Shirley\Downloads\BGL\result_full\BGL_TFIDF_preview.csv"
pd.DataFrame(X_tfidf[:100].toarray(), columns=vectorizer.get_feature_names_out()).to_csv(preview_csv, index=False)

print("✅ TF-IDF features saved:")
print("   Features file:", output_npz)
print("   Labels file:  BGL_labels.npy")
print("   Preview (first 100 samples):", preview_csv)


✅ Loaded data: (4643748, 6)
   TemplateID                                      EventTemplate   EventId  \
0           0  RAS KERNEL INFO instruction cache parity error...  b80f2820   
1           0  RAS KERNEL INFO instruction cache parity error...  b80f2820   
2           0  RAS KERNEL INFO instruction cache parity error...  b80f2820   

   Occurrences FirstColumn  Label  
0       105924           -      0  
1       105924           -      0  
2       105924           -      0  
✅ TF-IDF matrix shape: (4643748, 667)
✅ TF-IDF features saved:
   Features file: C:\Users\Shirley\Downloads\BGL\result_full\BGL_TFIDF_features.npz
   Labels file:  BGL_labels.npy
   Preview (first 100 samples): C:\Users\Shirley\Downloads\BGL\result_full\BGL_TFIDF_preview.csv
